In [ ]:
# Math
import numpy as np

# Visualization
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

plt.style.use('qualitative.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors_dict = {-1: colors[0], +1: colors[1]}

In [ ]:
# Load data
data = load_iris()
X = data['data'][:,[0,3]]
y = data['target']

# Change class labels to +1, -1
# Merge classes
y[y==1] = -1
y[y==0]=-1
y[y!=-1]=1

X -= np.mean(X, axis=0)

In [ ]:
def phi(x, y, d=2, total_ys=3):
    xhat = np.concatenate([x, np.array([1.])]) 
    return y*xhat

def loss(y1, y2):
    return (y1 != y2)

In [ ]:
def visualize_decision_boundary(w,
                                axis,
                                N: int = 1000,
                                xmin: float = np.min(X[:,0]),
                                xmax: float = np.max(X[:,0]),
                                ymin: float = np.min(X[:,1]),
                                ymax: float = np.max(X[:,1])):
    X = np.linspace(xmin, xmax, N)
    Y = np.linspace(ymin, ymax, N)
    X, Y = np.meshgrid(X, Y)
    def func(x):
        return np.dot(w, np.concatenate([x, np.ones([x.shape[0], 1])], axis=1).T)

    Z = func(np.c_[X.ravel(), Y.ravel()])
    Z = Z.reshape(X.shape)
    axis.contourf(X, Y, Z, levels=[-100, -1, 0, 1, 100], cmap=matplotlib.colors.ListedColormap(colors[:2]), alpha=0.5, linewidths=0)


In [ ]:
# 1) Choose an initial iterate w1
w = np.random.rand(X.shape[1]+1)

plt.scatter(X[:,0], X[:,1], c=[colors_dict[_] for _ in y])
visualize_decision_boundary(w, plt.gca())

plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title(f"Randomly Initialized SVM")
plt.show()

In [ ]:
number_of_iterations = 10000
reg = 0.0
step_size = 0.1

number_of_samples = X.shape[0]

# 2) For k = 1, 2, ...
for k in range(number_of_iterations):
    # 3) Select an example (x,y) uniformly at random
    selected_ix = np.random.randint(number_of_samples)
    selected_X = X[selected_ix, :]
    selected_y = y[selected_ix]

    # Naive implementation of Argmax(loss(y, y') + <w, phi(x, y')>)
    max_score = -np.infty
    predicted_y = None
    for possible_y in [-1, +1]:
        score = loss(selected_y, possible_y) + np.dot(w, phi(selected_X, possible_y))
        if score > max_score:
            max_score = score
            predicted_y = possible_y
    
    # 4) Compute subgradient
    g = reg*w \
        - phi(selected_X, selected_y) \
        + phi(selected_X, predicted_y)

    # 5) Set new iterate
    w -= step_size*g
    
plt.figure()
visualize_decision_boundary(w, plt.gca())
plt.scatter(X[:,0], X[:,1], c=[colors_dict[_] for _ in y])
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title(f"Trained SVM")
plt.show()
    